# 1 Overview volume timeseries plots (just for analysis)

---

In [1]:
DATE='apr04'
# download it here https://cluster.klima.uni-bremen.de/~lschuster/glacierMIP3_analysis/glacierMIP3_{DATE}_models_all_rgi_regions_sum_scaled.nc
# and change the path to your local path
path_merged_runs_scaled = f'/home/www/lschuster/glacierMIP3_analysis/glacierMIP3_{DATE}_models_all_rgi_regions_sum_scaled.nc'

---

In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
from datetime import date
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from help_func import pal_models, model_order, d_reg_num_name
hue_order = model_order
ds_reg_models = xr.open_dataset(path_merged_runs_scaled)

In [3]:
# drop Huss and OGGM_v153
models = ['James', 'Rounce', 'Compagno', 'Kraaijenbrink', 'GLIMB', 'OGGM_v16', 'OGGM-VAS', 'Zekollari']
ds_reg_models = ds_reg_models.sel(model_author = models)

In [6]:
# check if 0 removed from RGI region 17 , because there was some isssues in the GloGEM dataset 
ds_reg_models_sel = ds_reg_models.sel(rgi_reg='17')
assert len(ds_reg_models_sel.sel(simulation_year=2000).volume_m3.where(ds_reg_models_sel.sel(simulation_year=2000).volume_m3==0).dropna(dim='model_author', how='all').model_author) == 0
assert len(ds_reg_models_sel.sel(simulation_year=5000).volume_m3.where(ds_reg_models_sel.sel(simulation_year=5000).volume_m3==0).dropna(dim='model_author', how='all').model_author) == 0

## overview timeseries plots (just for analysis)

- time series for every experiment an other figure, another subplot for every region
- in every subplot: one RGI region, all the models and maybe all gcms?

In [7]:
plt.rc('font', size=20)     

In [8]:
rgi_regs = []
for rgi_reg in np.arange(1,20,1):
    if rgi_reg < 10:
        rgi_reg = '0'+str(rgi_reg)
    else:
        rgi_reg = str(rgi_reg)
    rgi_regs.append(rgi_reg)

In [9]:
ds_reg_models_stack = ds_reg_models.stack(exps =('gcm','rgi_reg',
                                                     'period_scenario', 'model_author'))
# test if all the experiments are unique
# for volume_m3
unq, count = np.unique(ds_reg_models_stack.volume_m3.values, axis=1, return_counts=True)
assert np.shape(ds_reg_models_stack.volume_m3.values) == np.shape(unq)
assert np.all(count==1)
# for area_m2
unq, count = np.unique(ds_reg_models_stack.area_m2.values, axis=1, return_counts=True)
assert np.shape(ds_reg_models_stack.area_m2.values) == np.shape(unq)
assert np.all(count==1)

In [11]:
pal_models

[(0.33725490196078434, 0.7058823529411765, 0.9137254901960784),
 (0.00392156862745098, 0.45098039215686275, 0.6980392156862745),
 (0.8705882352941177, 0.5607843137254902, 0.0196078431372549),
 (0.00784313725490196, 0.6196078431372549, 0.45098039215686275),
 (0.8352941176470589, 0.3686274509803922, 0.0),
 (0.8, 0.47058823529411764, 0.7372549019607844),
 (0.34901960784313724, 0.11764705882352941, 0.44313725490196076),
 (0.984313725490196, 0.6862745098039216, 0.8941176470588236),
 (0.5803921568627451, 0.5803921568627451, 0.5803921568627451),
 (0.9254901960784314, 0.8823529411764706, 0.2)]

In [15]:
dict_model_col = {}
for h, p in zip(hue_order,pal_models):
    if h in ds_reg_models.model_author:
        dict_model_col[h] = p

In [18]:
dict_model_col.values()

dict_values([(0.33725490196078434, 0.7058823529411765, 0.9137254901960784), (0.8705882352941177, 0.5607843137254902, 0.0196078431372549), (0.00784313725490196, 0.6196078431372549, 0.45098039215686275), (0.8352941176470589, 0.3686274509803922, 0.0), (0.8, 0.47058823529411764, 0.7372549019607844), (0.984313725490196, 0.6862745098039216, 0.8941176470588236), (0.5803921568627451, 0.5803921568627451, 0.5803921568627451), (0.9254901960784314, 0.8823529411764706, 0.2)])

In [19]:
ls_l = ['solid', 'dotted', 'dashed', 'dashdot', (0, (3, 1, 1, 1))]

for rgi_reg in rgi_regs:
    axs = []
    plt.figure(figsize=(30,40))
    for j,period_scenario in enumerate(ds_reg_models.period_scenario):
        if j==0 or j==4:
            ax0=plt.subplot(4,4,j+1)
            ax = ax0
        else:
            ax=plt.subplot(4,4,j+1,sharey=ax0)
        axs.append(ax)
        for ls,gcm in zip(ls_l,ds_reg_models.gcm):
            pd_reg_vol_sel = ds_reg_models.sel(period_scenario=period_scenario).sel(gcm=gcm).volume_m3.to_dataframe().reset_index()
            pd_reg_vol_sel['volume (km³)'] = pd_reg_vol_sel['volume_m3']/1e9
            if ls == 'solid' and j==3:
                sns.lineplot(x='simulation_year', data=pd_reg_vol_sel.loc[pd_reg_vol_sel.rgi_reg==rgi_reg], y='volume (km³)',
                         hue='model_author', hue_order=dict_model_col.keys(), palette=dict_model_col.values(), ls=ls, legend='brief', lw=3, ax=ax)
            else:
                sns.lineplot(x='simulation_year', data=pd_reg_vol_sel.loc[pd_reg_vol_sel.rgi_reg==rgi_reg], y='volume (km³)',
                         hue='model_author', hue_order=dict_model_col.keys(), palette=dict_model_col.values(), ls=ls, legend=False, lw=3, ax=ax)
            if j in [0,4,8,12]:
                ax.set_ylabel('volume (km³)', fontsize=22)
            else:
                ax.set_ylabel('')
            ax.set_xlabel('simulation year', fontsize=22)
            if ls == 'solid' and j==3: 
                leg = ax.get_legend()
                for legobj in leg.legendHandles:
                    legobj.set_linewidth(3.0)
            #leg.set_bbox_to_anchor([1,1])
        if j==3:
            minip, maxip = ax.get_ylim()
        elif j==15:
            minif, maxif = ax.get_ylim()
        
        ax.set_title(str(period_scenario.values))
        ax.set_xlim([-100,5100])
        
    for j,period_scenario in enumerate(ds_reg_models.period_scenario):
        ax = axs[j]
        ax2 = ax.twinx()
        for ls,gcm in zip(ls_l,ds_reg_models.gcm):
            _ref = 100*ds_reg_models.sel(period_scenario=period_scenario).sel(gcm=gcm).volume_m3/ds_reg_models.sel(period_scenario=period_scenario).sel(gcm=gcm).volume_m3.sel(simulation_year=0)
            pd_reg_vol_sel = _ref.to_dataframe().reset_index()
            pd_reg_vol_sel['volume (relative to initial state, in %)'] = pd_reg_vol_sel['volume_m3']
            if ls == 'solid' and j==3:
                sns.lineplot(x='simulation_year', data=pd_reg_vol_sel.loc[pd_reg_vol_sel.rgi_reg==rgi_reg], 
                             y='volume (relative to initial state, in %)',
                             hue='model_author', hue_order=dict_model_col.keys(), palette=dict_model_col.values(), ls=ls, legend='brief', lw=3, ax=ax2)
            else:
                sns.lineplot(x='simulation_year', data=pd_reg_vol_sel.loc[pd_reg_vol_sel.rgi_reg==rgi_reg], 
                             y='volume (relative to initial state, in %)',
                             hue='model_author', hue_order=dict_model_col.keys(), palette=dict_model_col.values(), ls=ls, legend=False, lw=3, ax=ax2)
            if j in [3,7,11,15]:
                ax2.set_ylabel('volume (relative to initial state, in %)', fontsize=22)
            else:
                ax2.set_ylabel('')
                
        rel = 100*1e9/ds_reg_models.sel(period_scenario=period_scenario).sel(gcm=gcm).sel(rgi_reg=rgi_reg).sel(simulation_year=0).volume_m3
        rel = rel.dropna(dim='model_author')
        np.testing.assert_allclose(rel[0], rel)
        rel = rel[0].values
        
        if j<4:
            ax2.set_ylim([minip*rel,maxip*rel])
        else:
            ax2.set_ylim([minif*rel,maxif*rel])

        
    reg = d_reg_num_name[rgi_reg]
    plt.suptitle(f'RGI region: {rgi_reg} ({reg})')
    plt.tight_layout()

    plt.savefig(f'figures/1_overview_timeseries_plots/vol_time_series_rgi_reg{rgi_reg}.png')
    plt.close()

- plots separately for every GCM:

In [20]:
ls = 'solid'
for gcm in ds_reg_models.gcm.values:

    for rgi_reg in rgi_regs:
        plt.figure(figsize=(30,40))
        axs = []
        for j,period_scenario in enumerate(ds_reg_models.period_scenario):
            if j==0 or j==4:
                ax0=plt.subplot(4,4,j+1)
                ax = ax0
            else:
                ax=plt.subplot(4,4,j+1,sharey=ax0)
            axs.append(ax)
            pd_reg_vol_sel = ds_reg_models.sel(period_scenario=period_scenario).sel(gcm=gcm).volume_m3.to_dataframe().reset_index()
            pd_reg_vol_sel['volume (km³)'] = pd_reg_vol_sel['volume_m3']/1e9
            if j==3:
                sns.lineplot(x='simulation_year', data=pd_reg_vol_sel.loc[pd_reg_vol_sel.rgi_reg==rgi_reg], y='volume (km³)',
                         hue='model_author', hue_order=dict_model_col.keys(), palette=dict_model_col.values(), ls=ls, legend='brief', lw=3, ax=ax)
            else:
                sns.lineplot(x='simulation_year', data=pd_reg_vol_sel.loc[pd_reg_vol_sel.rgi_reg==rgi_reg], y='volume (km³)',
                         hue='model_author', hue_order=dict_model_col.keys(), palette=dict_model_col.values(), ls=ls, legend=False, lw=3, ax=ax)
            if j in [0,4,8,12]:
                plt.ylabel('volume (km³)', fontsize=22)
            else:
                plt.ylabel('')
            plt.xlabel('simulation year', fontsize=22)
            if j==3: 
                leg = plt.gca().get_legend()
                for legobj in leg.legendHandles:
                    legobj.set_linewidth(3.0)
            #leg.set_bbox_to_anchor([1,1])
            ax.set_title(str(period_scenario.values))
            ax.set_xlim([-100,5100])
            if j==3:
                minip, maxip = ax.get_ylim()
            elif j==15:
                minif, maxif = ax.get_ylim()


        
        for j,period_scenario in enumerate(ds_reg_models.period_scenario):
            ax = axs[j]
            ax2 = ax.twinx()
            _ref = 100*ds_reg_models.sel(period_scenario=period_scenario).sel(gcm=gcm).volume_m3/ds_reg_models.sel(period_scenario=period_scenario).sel(gcm=gcm).volume_m3.sel(simulation_year=0)
            pd_reg_vol_sel = _ref.to_dataframe().reset_index()
            pd_reg_vol_sel['volume (relative to initial state, in %)'] = pd_reg_vol_sel['volume_m3']
            if ls == 'solid' and j==3:
                sns.lineplot(x='simulation_year', data=pd_reg_vol_sel.loc[pd_reg_vol_sel.rgi_reg==rgi_reg], 
                             y='volume (relative to initial state, in %)',
                             hue='model_author', hue_order=dict_model_col.keys(), palette=dict_model_col.values(), ls=ls, legend='brief', lw=3, ax=ax2)
            else:
                sns.lineplot(x='simulation_year', data=pd_reg_vol_sel.loc[pd_reg_vol_sel.rgi_reg==rgi_reg], 
                             y='volume (relative to initial state, in %)',
                             hue='model_author', hue_order=dict_model_col.keys(), palette=dict_model_col.values(), ls=ls, legend=False, lw=3, ax=ax2)
            if j in [3,7,11,15]:
                ax2.set_ylabel('volume (relative to initial state, in %)', fontsize=22)
            else:
                ax2.set_ylabel('')
    
                
            rel = 100*1e9/ds_reg_models.sel(period_scenario=period_scenario).sel(gcm=gcm).sel(rgi_reg=rgi_reg).sel(simulation_year=0).volume_m3
            rel = rel.dropna(dim='model_author')
            np.testing.assert_allclose(rel[0], rel)
            rel = rel[0].values

            if j<4:
                ax2.set_ylim([minip*rel,maxip*rel])
            else:
                ax2.set_ylim([minif*rel,maxif*rel])

        reg = d_reg_num_name[rgi_reg]
        plt.suptitle(f'RGI region: {rgi_reg} ({reg}) - {gcm}')
        plt.tight_layout()

        plt.savefig(f'figures/1_overview_timeseries_plots/gcm_separate_vol_time_series/vol_time_series_rgi_reg{rgi_reg}_{gcm}.png')
        plt.close()

- old: same plot again without UKESM

In [ ]:
old = False
if old:
    ls_l = ['solid', 'dotted', 'dashed', 'dashdot', (0, (3, 1, 1, 1))]

    for rgi_reg in rgi_regs:
        axs = []
        plt.figure(figsize=(30,40))
        for j,period_scenario in enumerate(ds_reg_models.period_scenario):
            if j==0 or j==4:
                ax0=plt.subplot(4,4,j+1)
                ax = ax0
            else:
                ax=plt.subplot(4,4,j+1,sharey=ax0)
            axs.append(ax)
            for ls,gcm in zip(ls_l,['gfdl-esm4', 'ipsl-cm6a-lr', 'mpi-esm1-2-hr', 'mri-esm2-0']):
                pd_reg_vol_sel = ds_reg_models.sel(period_scenario=period_scenario).sel(gcm=gcm).volume_m3.to_dataframe().reset_index()
                pd_reg_vol_sel['volume (km³)'] = pd_reg_vol_sel['volume_m3']/1e9
                if ls == 'solid' and j==3:
                    sns.lineplot(x='simulation_year', data=pd_reg_vol_sel.loc[pd_reg_vol_sel.rgi_reg==rgi_reg], y='volume (km³)',
                             hue='model_author', hue_order=hue_order, palette=pal_models, ls=ls, legend='brief', lw=3, ax=ax)
                else:
                    sns.lineplot(x='simulation_year', data=pd_reg_vol_sel.loc[pd_reg_vol_sel.rgi_reg==rgi_reg], y='volume (km³)',
                             hue='model_author', hue_order=hue_order, palette=pal_models, ls=ls, legend=False, lw=3, ax=ax)
                if j in [0,4,8,12]:
                    ax.set_ylabel('volume (km³)', fontsize=22)
                else:
                    ax.set_ylabel('')
                ax.set_xlabel('simulation year', fontsize=22)
                if ls == 'solid' and j==3: 
                    leg = ax.get_legend()
                    for legobj in leg.legendHandles:
                        legobj.set_linewidth(3.0)
                #leg.set_bbox_to_anchor([1,1])
            if j==3:
                minip, maxip = ax.get_ylim()
            elif j==15:
                minif, maxif = ax.get_ylim()

            ax.set_title(str(period_scenario.values))
            ax.set_xlim([-100,5100])

        for j,period_scenario in enumerate(ds_reg_models.period_scenario):
            ax = axs[j]
            ax2 = ax.twinx()
            for ls,gcm in zip(ls_l, ['gfdl-esm4', 'ipsl-cm6a-lr', 'mpi-esm1-2-hr', 'mri-esm2-0']):
                _ref = 100*ds_reg_models.sel(period_scenario=period_scenario).sel(gcm=gcm).volume_m3/ds_reg_models.sel(period_scenario=period_scenario).sel(gcm=gcm).volume_m3.sel(simulation_year=0)
                pd_reg_vol_sel = _ref.to_dataframe().reset_index()
                pd_reg_vol_sel['volume (relative to initial state, in %)'] = pd_reg_vol_sel['volume_m3']
                if ls == 'solid' and j==3:
                    sns.lineplot(x='simulation_year', data=pd_reg_vol_sel.loc[pd_reg_vol_sel.rgi_reg==rgi_reg], 
                                 y='volume (relative to initial state, in %)',
                                 hue='model_author', hue_order=hue_order, palette=pal_models, ls=ls, legend='brief', lw=3, ax=ax2)
                else:
                    sns.lineplot(x='simulation_year', data=pd_reg_vol_sel.loc[pd_reg_vol_sel.rgi_reg==rgi_reg], 
                                 y='volume (relative to initial state, in %)',
                                 hue='model_author', hue_order=hue_order, palette=pal_models, ls=ls, legend=False, lw=3, ax=ax2)
                if j in [3,7,11,15]:
                    ax2.set_ylabel('volume (relative to initial state, in %)', fontsize=22)
                else:
                    ax2.set_ylabel('')

            rel = 100*1e9/ds_reg_models.sel(period_scenario=period_scenario).sel(gcm=gcm).sel(rgi_reg=rgi_reg).sel(simulation_year=0).volume_m3
            rel = rel.dropna(dim='model_author')
            np.testing.assert_allclose(rel[0], rel)
            rel = rel[0].values

            if j<4:
                ax2.set_ylim([minip*rel,maxip*rel])
            else:
                ax2.set_ylim([minif*rel,maxif*rel])


        reg = d_reg_num_name[rgi_reg]
        plt.suptitle(f'RGI region: {rgi_reg} ({reg})')
        plt.tight_layout()

        plt.savefig(f'/home/www/lschuster/glacierMIP3_analysis/analysis_2023_03/no_ukesm/vol_time_series_rgi_reg{rgi_reg}.pdf')
        plt.close()